<a href="https://colab.research.google.com/github/roshni33/100-days-of-AI/blob/main/movie_recommendation_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#importing libraries

import numpy as np
import pandas as pd
import difflib
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
#importing dataset

df=pd.read_csv('/content/movies.csv')

<ipython-input-2-3f710e30d4a0>:1: DtypeWarning: Columns (0,1,4,9,13,14,19,20,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273

In [ ]:
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]


In [ ]:
df.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.31295,...,165,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [ ]:
#total rows,scols
df.shape

(4815, 24)

Feature Selection - Selecting only top 5 important features

In [ ]:
selected_features=['genres','keywords','tagline','cast','director']
print(selected_features)

['genres', 'keywords', 'tagline', 'cast', 'director']


Preprocessing selected features

In [ ]:
#replace Null values with null string

for feature in selected_features:
  df[feature]=df[feature].fillna('')

In [ ]:
#combining all 5 features

combined_features=df['genres']+df['keywords']+df['tagline']+df['cast']+df['director']
print(combined_features)

0       Action Adventure Fantasy Science Fictioncultur...
1       Adventure Fantasy Actionocean drug abuse exoti...
2       Action Adventure Crimespy based on novel secre...
3       Action Crime Drama Thrillerdc comics crime fig...
4       Action Adventure Science Fictionbased on novel...
                              ...                        
4810    Action Crime Thrillerunited states\u2013mexico...
4811    Comedy RomanceA newlywed couple's honeymoon is...
4812    Comedy Drama Romance TV Moviedate love at firs...
4813    A New Yorker in ShanghaiDaniel Henney Eliza Co...
4814    Documentaryobsession camcorder crush dream gir...
Length: 4815, dtype: object


Feature Extraction

In [ ]:
vectorizer=TfidfVectorizer()

In [ ]:
feature_vectors = vectorizer.fit_transform(combined_features)
print(feature_vectors)

  (0, 251)	0.07863237367861305
  (0, 451)	0.09113514354042858
  (0, 8941)	0.11809220364547982
  (0, 21839)	0.09970411878476632
  (0, 9309)	0.2738269443320904
  (0, 4461)	0.21798797231603356
  (0, 10198)	0.1653364514573188
  (0, 22919)	0.3367094215613518
  (0, 26338)	0.1306231420758433
  (0, 4680)	0.2452713633992677
  (0, 22781)	0.2738269443320904
  (0, 24201)	0.07523334570513966
  (0, 27165)	0.12622652297847456
  (0, 18169)	0.08695157064352238
  (0, 18563)	0.26119089847103877
  (0, 21525)	0.15625140137358118
  (0, 27186)	0.23478543371532049
  (0, 27544)	0.19771285083703202
  (0, 21491)	0.21549349546764407
  (0, 22467)	0.2020684372442625
  (0, 26551)	0.196386509238531
  (0, 23270)	0.161295129800485
  (0, 14555)	0.2236699003217651
  (0, 16619)	0.15697618575333463
  (0, 20733)	0.2738269443320904
  :	:
  (4813, 18795)	0.22451821435288907
  (4813, 11781)	0.3005525904690098
  (4813, 765)	0.1937259098191877
  (4813, 8126)	0.26115545962362025
  (4813, 27517)	0.3005525904690098
  (4813, 27365)	

Finding Cosine Similarity

In [ ]:
similarity=cosine_similarity(feature_vectors)
print(similarity)

[[1.         0.06870881 0.01493791 ... 0.         0.         0.        ]
 [0.06870881 1.         0.02801131 ... 0.0124378  0.         0.        ]
 [0.01493791 0.02801131 1.         ... 0.         0.         0.        ]
 ...
 [0.         0.0124378  0.         ... 1.         0.         0.        ]
 [0.         0.         0.         ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.         0.         1.        ]]


In [ ]:
#user i/p -
movie_name=input("enter movie name - ")

enter movie name - iron man


In [ ]:
list_of_all_titles = df['title'].dropna().tolist()
print(list_of_all_titles)

['Avatar', "Pirates of the Caribbean: At World's End", 'Spectre', 'The Dark Knight Rises', 'John Carter', 'Spider-Man 3', 'Tangled', 'Avengers: Age of Ultron', 'Harry Potter and the Half-Blood Prince', 'Batman v Superman: Dawn of Justice', 'Superman Returns', 'Quantum of Solace', "Pirates of the Caribbean: Dead Man's Chest", 'The Lone Ranger', 'Man of Steel', 'The Chronicles of Narnia: Prince Caspian', 'The Avengers', 'Pirates of the Caribbean: On Stranger Tides', 'Men in Black 3', 'The Hobbit: The Battle of the Five Armies', 'The Amazing Spider-Man', 'Robin Hood', 'The Hobbit: The Desolation of Smaug', 'The Golden Compass', 'King Kong', 'Titanic', 'Captain America: Civil War', 'Battleship', 'Jurassic World', " 'department': 'Sound'", 'Skyfall', 'Spider-Man 2', 'Iron Man 3', 'Alice in Wonderland', 'X-Men: The Last Stand', 'Monsters University', 'Transformers: Revenge of the Fallen', 'Transformers: Age of Extinction', 'Oz: The Great and Powerful', 'The Amazing Spider-Man 2', 'TRON: Lega

In [ ]:
# finding the close match for the movie name given by the user

find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)
print(find_close_match)

['Iron Man', 'Iron Man 3', 'Iron Man 2']


In [ ]:
#finding index of user i/p

index_of_movie=df[df.title == find_close_match[0]]['index'].values[0]
print(index_of_movie)

68


In [ ]:
index_of_movie = int(index_of_movie)  # Ensure it's an integer # otherwise you encounter an error


In [ ]:
#Finding similarity between user inputted movie and every other movie in the dataset

similarity_score=list(enumerate(similarity[index_of_movie]))
print(similarity_score)

[(0, 0.0053840453964320205), (1, 0.01412105258668815), (2, 0.0), (3, 0.0), (4, 0.021391405248879804), (5, 0.0), (6, 0.0319755152886171), (7, 0.0), (8, 0.0), (9, 0.0), (10, 0.006220470878064989), (11, 0.0), (12, 0.0), (13, 0.0052167793460599745), (14, 0.005476497113605312), (15, 0.011776531223683506), (16, 0.0), (17, 0.0), (18, 0.020845165260144363), (19, 0.009445648827897203), (20, 0.0), (21, 0.019234633021407675), (22, 0.005152876459122566), (23, 0.009292300519067664), (24, 0.005056638581707381), (25, 0.0), (26, 0.0), (27, 0.0), (28, 0.0), (29, 0.0), (30, 0.0), (31, 0.0), (32, 0.010921499259123202), (33, 0.01070674082737502), (34, 0.021297047131284795), (35, 0.0), (36, 0.06304429905707336), (37, 0.0), (38, 0.0), (39, 0.0), (40, 0.0), (41, 0.0), (42, 0.05685801540235139), (43, 0.0), (44, 0.042616670155936294), (45, 0.005422630453723399), (46, 0.0), (47, 0.0), (48, 0.01013965163177446), (49, 0.0), (50, 0.013161248717525083), (51, 0.004739694559056603), (52, 0.0054875562161488135), (53, 

In [ ]:
#Descending order with respect to their similarity score

sorted_similar_movies=sorted(similarity_score,key=lambda x:x[1], reverse=True)
print(sorted_similar_movies)                #o/p - (68, 1.0) means (index,similiarty_score)

[(68, 1.0), (79, 0.13397088475483065), (295, 0.12729460080683605), (3682, 0.10753570100180734), (1748, 0.10258813594489788), (3432, 0.10099682735707477), (944, 0.09530596291376309), (2757, 0.09254187957086377), (4615, 0.0874663805044487), (1111, 0.08323519249761464), (3126, 0.08202929361718031), (3415, 0.08182746035298587), (1629, 0.08149835833215835), (366, 0.07897201278267808), (3248, 0.07745967839326438), (2339, 0.07627299520375581), (260, 0.07614265920897502), (3512, 0.07534674860217058), (4564, 0.07516125283985743), (3099, 0.07458819843005124), (2128, 0.07439095567933819), (3681, 0.07413839411046329), (655, 0.07316950145187501), (3366, 0.07065364842323507), (588, 0.06875047751591043), (462, 0.06841837745019892), (240, 0.06828390493743115), (1405, 0.06777798743062313), (4680, 0.06776484436987494), (906, 0.06740680840541269), (3564, 0.06711480336020031), (2379, 0.06691648800133787), (189, 0.06679459596530071), (94, 0.06669328143017339), (2080, 0.06517787068851191), (1473, 0.06465500

In [ ]:
print("Top 10 suggested movies are - ")
i=1
for movie in sorted_similar_movies:
  index=movie[0]
  title_from_index=df[df.index == index]['title'].values[0]
  if i<10:
    print(i,'.',title_from_index)
    i=i+1

Top 10 suggested movies are - 
1 . Up
2 . Inside Out
3 . The Emperor's New Groove
4 . Running Forever
5 . Our Brand Is Crisis
6 . Boyz n the Hood
7 . Curious George
8 . This Christmas
9 . My Dog Tulip


In [ ]:
#latest code - ironman,iron man + o/p include iron man,iron man 2 , iron man 3

import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import difflib

# Load the dataset and clean it
df = pd.read_csv('/content/movies.csv')
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
selected_features = ['genres', 'keywords', 'tagline', 'cast', 'director']

# Replace NaN values with empty strings
for feature in selected_features:
    df[feature] = df[feature].fillna('')

# Combine selected features into a single text-based feature
combined_features = df['genres'] + ' ' + df['keywords'] + ' ' + df['tagline'] + ' ' + df['cast'] + ' ' + df['director']

# Convert text to feature vectors using TF-IDF
vectorizer = TfidfVectorizer()
feature_vectors = vectorizer.fit_transform(combined_features)

# Calculate similarity matrix
similarity = cosine_similarity(feature_vectors)

# User input movie name
movie_name = input("Enter movie name - ").lower()

# Ensure the 'title' column has no null values and remove them
df['title'] = df['title'].fillna('')  # Replace NaN in title column with empty strings

# Normalize movie titles by removing spaces for matching purposes
df['normalized_title'] = df['title'].str.replace(' ', '').str.lower()

# Normalize user input movie name by removing spaces
normalized_movie_name = movie_name.replace(' ', '')

# Find all movie titles and find close matches for the normalized input movie
list_of_all_titles = df['normalized_title'].tolist()  # Convert titles to lowercase without spaces

# Find the closest match for the user's normalized movie name
find_close_match = difflib.get_close_matches(normalized_movie_name, list_of_all_titles)

# Handle the case where no close match is found
if not find_close_match:
    print(f"No match found for the movie {movie_name}")
else:
    # Find the index of the first close match (assume it's the best match)
    close_match_title = find_close_match[0]
    movie_index = df[df['normalized_title'] == close_match_title].index[0]  # Get the row index

    # Get similarity scores for all movies
    similarity_score = list(enumerate(similarity[int(movie_index)]))

    # Sort movies based on similarity scores
    sorted_similar_movies = sorted(similarity_score, key=lambda x: x[1], reverse=True)

    # Create a list to hold movie titles in the recommendations
    recommended_movies = []

    # Start by including the user-input movie and its sequels (if available)
    user_movie_series = []
    for title in list_of_all_titles:
        if normalized_movie_name in title:  # Find all movies that contain the normalized input movie name
            user_movie_series.append(df[df['normalized_title'] == title]['title'].values[0])

    # Add user movie series (e.g., Iron Man, Iron Man 2, Iron Man 3) at the top
    recommended_movies.extend(user_movie_series)

    # Add other similar movies to fill up the top 10 suggestions
    for movie in sorted_similar_movies:
        index = movie[0]
        title_from_index = df['title'].iloc[index]

        # Avoid adding duplicates from the user movie series
        if title_from_index not in recommended_movies:
            recommended_movies.append(title_from_index)

        # Stop when the top 10 recommendations are collected
        if len(recommended_movies) >= 10:
            break

    # Display top 10 recommended movies
    print("Top 10 suggested movies are -")
    for i, title in enumerate(recommended_movies[:10], 1):
        print(f"{i}. {title}")


<ipython-input-76-8ce68c97ee7c>:8: DtypeWarning: Columns (0,1,4,9,13,14,19,20,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,27

Enter movie name - ironman
Top 10 suggested movies are -
1. Iron Man 3
2. Iron Man
3. Iron Man 2
4. Avengers: Age of Ultron
5. The Avengers
6. Captain America: Civil War
7. Captain America: The Winter Soldier
8. Ant-Man
9. X-Men
10. Made
